In [1]:
import datajoint as dj
import numpy as np
import time
from tqdm import tqdm

In [2]:
#setting the address and the username
dj.config['database.host'] = '10.28.0.34'
dj.config['database.user'] = 'celiib'
dj.config['database.password'] = 'newceliipass'
dj.config['safemode']=True
dj.config["display.limit"] = 20

ta3p100 = dj.create_virtual_module('ta3p100', 'microns_ta3p100')
schema = dj.schema('microns_ta3p100')


Connecting celiib@10.28.0.34:3306


In [8]:
@schema
class CoarseLabel(dj.Computed):
    definition = """
    # Vertex labels for ta3p100.ProofreadLabel did not correctly match the triangle labels, so these are regenerated from the correct triangle labels.
    -> ta3p100.ProofreadLabel
    ---
    vertices  : longblob # Corrected vertex labels
    triangles : longblob # Same triangle labels as ta3p100.ProofreadLabel
    """
    
    key_source = ta3p100.ProofreadLabel & 'status="complete"'
    
    def make(self, key):
        start = time.time()
        
        print(key["segment_id"])
        labels = (ta3p100.ProofreadLabel & key).fetch1()
        corrected_vertex_labels = np.zeros(labels['vertices'].shape, np.uint8)
        
        mesh = (ta3p100.CleansedMesh & 'decimation_ratio=0.35' & dict(segment_id=key['segment_id'])).fetch1()
        mesh_triangles = mesh['triangles']
        
        vertex_label_dict = dict()
        for i, label in enumerate(labels['triangles']):
            triangle = mesh_triangles[i]
            for node in triangle:
                if node in vertex_label_dict:
                    if vertex_label_dict[node] < label:
                        vertex_label_dict[node] = label
                else:
                    vertex_label_dict[node] = label
                
        for node, label in vertex_label_dict.items():
            corrected_vertex_labels[node] = label
            
        self.insert1(dict(key,
                          vertices=corrected_vertex_labels,
                          triangles=labels['triangles']))
        
        print("Segment {} vertex labels regenerated in: {} seconds.".format(key['segment_id'], time.time() - start))

In [9]:
start = time.time()
CoarseLabel.populate(reserve_jobs=True)
print(time.time() - start)

648518346341366885
Segment 648518346341366885 vertex labels regenerated in: 20.977378129959106 seconds.
648518346341371119
Segment 648518346341371119 vertex labels regenerated in: 3.396214008331299 seconds.
648518346341388820
Segment 648518346341388820 vertex labels regenerated in: 2.9767889976501465 seconds.
648518346341393609
Segment 648518346341393609 vertex labels regenerated in: 4.248183965682983 seconds.
648518346342797556
Segment 648518346342797556 vertex labels regenerated in: 3.3691937923431396 seconds.
648518346342806978
Segment 648518346342806978 vertex labels regenerated in: 3.4192819595336914 seconds.
648518346349386137
Segment 648518346349386137 vertex labels regenerated in: 2.0592539310455322 seconds.
648518346349468264
Segment 648518346349468264 vertex labels regenerated in: 4.25806999206543 seconds.
648518346349470171
Segment 648518346349470171 vertex labels regenerated in: 3.4856321811676025 seconds.
648518346349471156
Segment 648518346349471156 vertex labels regenera

648518346349499876
Segment 648518346349499876 vertex labels regenerated in: 5.017797946929932 seconds.
648518346349499928
Segment 648518346349499928 vertex labels regenerated in: 3.0710020065307617 seconds.
648518346349500120
Segment 648518346349500120 vertex labels regenerated in: 3.7147910594940186 seconds.
648518346349500139
Segment 648518346349500139 vertex labels regenerated in: 5.07932186126709 seconds.
648518346349500162
Segment 648518346349500162 vertex labels regenerated in: 3.764157772064209 seconds.
648518346349500277
Segment 648518346349500277 vertex labels regenerated in: 4.63598108291626 seconds.
648518346349500341
Segment 648518346349500341 vertex labels regenerated in: 4.355835676193237 seconds.
648518346349500440
Segment 648518346349500440 vertex labels regenerated in: 5.233341217041016 seconds.
648518346349500657
Segment 648518346349500657 vertex labels regenerated in: 6.917334794998169 seconds.
648518346349500742
Segment 648518346349500742 vertex labels regenerated i

In [10]:
ta3p100.CoarseLabel()

segmentation segmentation id,segment_id segment id unique within each Segmentation,decimation_ratio,author_original name of last editor,author_proofreader name of last editor,date_time the last time it was edited,vertices Corrected vertex labels,triangles Same triangle labels as ta3p100.ProofreadLabel
2,648518346341366885,0.35,computer_Auto,bcelii,2019-01-24 20:35:07,=BLOB=,=BLOB=
2,648518346341371119,0.35,computer_Auto,zhiweid,2019-01-24 01:36:06,=BLOB=,=BLOB=
2,648518346341388820,0.35,computer_Auto,zhiweid,2019-01-26 14:59:55,=BLOB=,=BLOB=
2,648518346341393609,0.35,computer_Auto,bcelii,2019-01-24 19:36:28,=BLOB=,=BLOB=
2,648518346342797556,0.35,computer_Auto,bcelii,2019-01-24 18:49:45,=BLOB=,=BLOB=
2,648518346342806978,0.35,computer_Auto,bcelii,2019-01-24 18:44:51,=BLOB=,=BLOB=
2,648518346349386137,0.35,computer_Auto,bcelii,2019-01-24 12:25:57,=BLOB=,=BLOB=
2,648518346349468264,0.35,computer_Auto,spapadop,2019-01-25 14:20:05,=BLOB=,=BLOB=
2,648518346349470171,0.35,computer_Auto,spapadop,2019-01-25 14:22:26,=BLOB=,=BLOB=
2,648518346349471156,0.35,computer_Auto,spapadop,2019-01-24 10:35:53,=BLOB=,=BLOB=


In [7]:
(schema.jobs & "table_name='__coarse_label'").delete()